# Naive Bayes - The probabilistic model
## An special case of the Gausian Discriminant Analysis
Not the GDA but it is discrete use of the GDA with the bayes assumption


In [5]:
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.phi_0 = []       # P(x|y=0)
        self.phi_1 = []       # P(x|y=1)
        self.phi_y = {}       # P(y)
        self.features = None  # Unique feature values
    
    def fit(self, X, y):
        self.features = np.unique(X)
        V = len(self.features) 
        
        # Class priors
        self.phi_y[0] = np.sum(y == 0) / len(y)
        self.phi_y[1] = np.sum(y == 1) / len(y)


        X0 = X[y == 0]
        X1 = X[y == 1]

        features0, counts0 = np.unique(X0, return_counts=True)
        features1, counts1 = np.unique(X1, return_counts=True)

        # Apply Laplace smoothing
        counts0 = counts0 + 1
        counts1 = counts1 + 1

        denom0 = len(X0) + V
        denom1 = len(X1) + V

        for feat in self.features:
            # For y = 0
            if feat in features0:
                idx = self._idx(feat, features0)
                self.phi_0.append(counts0[idx] / denom0)
            else:
                self.phi_0.append(1 / denom0) 

            # For y = 1
            if feat in features1:
                idx = self._idx(feat, features1)
                self.phi_1.append(counts1[idx] / denom1)
            else:
                self.phi_1.append(1 / denom1)

    def _idx(self, value, arr):
        for i in range(len(arr)):
            if arr[i] == value:
                return i
        return -1 

    def predict(self, x):
        p0 = self.phi_y[0]
        p1 = self.phi_y[1]

        for feat in x:
            if feat in self.features:
                idx = self._idx(feat, self.features)
                p0 *= self.phi_0[idx]
                p1 *= self.phi_1[idx]
            else:
                V = len(self.features)
                total0 = sum(self.phi_0) * (len(self.features)) 
                total1 = sum(self.phi_1) * (len(self.features))
                p0 *= 1 / (total0 + V)
                p1 *= 1 / (total1 + V)

        return 0 if p0 > p1 else 1
